In [2]:
import os, sys, shutil, subprocess
from pathlib import Path

os.environ['PYTHONNOUSERSITE'] = '1'
os.environ.pop('PIP_TARGET', None)

# Aggressive global uninstall of CPU torch to avoid interference
print('Uninstalling global torch...')
subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', 'torch', 'torchvision', 'torchaudio'], check=False)

# Clean stray site dirs that can shadow correct wheels (idempotent)
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch',
):
    if os.path.exists(d):
        print('Removing', d)
        shutil.rmtree(d, ignore_errors=True)

# Create GPU venv
venv_gpu = '/app/venv_gpu'
if os.path.exists(venv_gpu):
    shutil.rmtree(venv_gpu)
subprocess.run([sys.executable, '-m', 'venv', venv_gpu], check=True)
vpy = f'{venv_gpu}/bin/python'
vpip = f'{venv_gpu}/bin/pip'
subprocess.run([vpy, '-m', 'pip', 'install', '--upgrade', 'pip', 'setuptools', 'wheel'], check=True)

def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([vpy, '-m', 'pip', *args], check=True)

# 0) Hard reset any prior torch stacks in venv
for pkg in ('torch','torchvision','torchaudio'):
    subprocess.run([vpy, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

# 1) Install the EXACT cu121 torch stack FIRST (matches your CUDA 12.1 container) with force-reinstall
pip('install', '--force-reinstall',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# 2) Create a constraints file to freeze torch versions for all later installs
Path('constraints.txt').write_text(
    'torch==2.4.1\n'
    'torchvision==0.19.1\n'
    'torchaudio==2.4.1\n')

# 3) Now install NON-torch deps, honoring constraints, and avoid upgrading torch
pip('install', '-c', 'constraints.txt',
    'transformers==4.44.2', 'accelerate==0.34.2',
    'datasets==2.21.0', 'evaluate==0.4.2',
    'sentencepiece', 'scikit-learn', 'psutil',
    '--upgrade-strategy', 'only-if-needed')

# Activate venv path
os.environ['PATH'] = f'{venv_gpu}/bin:' + os.environ.get('PATH', '')
sys.path.insert(0, f'{venv_gpu}/lib/python{sys.version_info.major}.{sys.version_info.minor}/site-packages')

print('GPU venv setup complete. Restart kernel for full activation...')
os._exit(0)

Uninstalling global torch...


Found existing installation: torch 2.4.1+cpu


Uninstalling torch-2.4.1+cpu:
  Successfully uninstalled torch-2.4.1+cpu
Found existing installation: torchvision 0.19.1+cpu
Uninstalling torchvision-0.19.1+cpu:
  Successfully uninstalled torchvision-0.19.1+cpu
Found existing installation: torchaudio 2.4.1+cpu
Uninstalling torchaudio-2.4.1+cpu:
  Successfully uninstalled torchaudio-2.4.1+cpu


> install --force-reinstall --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━ 23/26 [torch]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━ 24/26 [torchvision]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26/26 [torchaudio]


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 0.30.1 requires psutil, which is not installed.
datasets 2.21.0 requires fsspec[http]<=2024.6.1,>=2023.1.0, but you have fsspec 2025.9.0 which is incompatible.


> install -c constraints.txt transformers==4.44.2 accelerate==0.34.2 datasets==2.21.0 evaluate==0.4.2 sentencepiece scikit-learn psutil --upgrade-strategy only-if-needed
